# はじめる

このノートブックを実行する前に、次のことを確認してください。

1。**ローカルモジュール：**必要なローカルPythonファイル（ `items.py`、` loaders.py`、 `test.py`）をColab Runtimeの一時ストレージにアップロードします。これを行うには、左サイドバーのフォルダーアイコン、次にアップロードアイコンをクリックして、ファイルを選択します。
2。左側のサイドバーのキーアイコンをクリックし、[新しいシークレット]をクリックし、「hf_token」という名前のトークンを追加します。
3。

これらの手順が完了したら、残りのノートブックセルを順番に実行できます。

In [ ]:
# コースの環境に合わせて、ローカル環境から正確なバージョンをインストールする
!pip install --upgrade pip

# 必要なライブラリの特定のバージョンをインストールします
!pip install datasets==3.6.0
!pip install transformers==4.51.3
!pip install huggingface_hub==0.31.2
!pip install matplotlib==3.10.3
!pip install numpy==1.26.4
!pip install python-dotenv==1.1.0
!pip install tqdm==4.67.1

In [ ]:
# 必要なライブラリをインポートします
import os
import random
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import numpy as np
import pickle

In [ ]:
# Colabのユーザーデータの秘密からハグするフェイスアクセストークンを取得する
# このトークンは、ハグする顔のハブと対話するために必要です
from google.colab import userdata
userdata.get('HF_TOKEN')

In [ ]:
# ローカルファイルからカスタムクラスをインポートする（items.pyおよびloaders.py）
# これらのファイルは、Colab Runtimeの一時ストレージに手動で追加されました
from loaders import ItemLoader
from items import Item

In [ ]:
# Matplotlibのバックエンドを設定して、ノートブックにインラインでプロットを表示する
%matplotlib inline

In [ ]:
# カスタムアイテムローダーを使用して、単一のデータセット（ "all_beauty"）をロードします
# これはおそらく最初のテストまたは例の読み込みステップでした
items = ItemLoader("Appliances").load()

In [ ]:
# ロードされるデータセット名（Amazon製品カテゴリ）のリストを定義する
dataset_names = [
    "Automotive",
    "Electronics",
    "Office_Products",
    "Tools_and_Home_Improvement",
    "Cell_Phones_and_Accessories",
    "Toys_and_Games",
    "Appliances",
    "Musical_Instruments",
]

In [ ]:
# コラブランタイム環境で利用可能なCPUコアとRAMを確認して印刷します
# これにより、データ処理に利用できるリソースを理解するのに役立ちます
import psutil
print(f"CPU cores: {psutil.cpu_count()}")
print(f"Available RAM: {psutil.virtual_memory().available / (1024**3):.1f} GB")

In [ ]:
items = []
for dataset_name in dataset_names:
    loader = ItemLoader(dataset_name)
    items.extend(loader.load(workers=8))

# さて、コーヒーブレイクの時間!!
# ちなみに、私は最大のデータセットを最初に置きます。それはより速くなります。

In [ ]:
# すべてのデータセットからロードされたアイテムの総数を印刷します
print(f"A grand total of {len(items):,} items")

In [ ]:
# すべてのロードされたアイテムからトークンカウントを抽出します
tokens = [item.token_count for item in items]
# トークンカウントのヒストグラムを作成して表示します
plt.figure(figsize=(15, 6))
plt.title(f"Token counts: Avg {sum(tokens)/len(tokens):,.1f} and highest {max(tokens):,}\n")
plt.xlabel('Length (tokens)')
plt.ylabel('Count')
plt.hist(tokens, rwidth=0.7, color="skyblue", bins=range(0, 300, 10))
plt.show()

In [ ]:
# すべてのロードされたアイテムから価格を抽出します
prices = [item.price for item in items]
# アイテム価格のヒストグラムを作成して表示します
plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.1f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="blueviolet", bins=range(0, 1000, 10))
plt.show()

In [ ]:
# ロードされたアイテムの各カテゴリの発生を数えます
category_counts = Counter()
for item in items:
    category_counts[item.category]+=1

# カテゴリとプロットのカウントを抽出します
categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

# カテゴリごとのアイテムのカウントを示すバーチャートを作成して表示します
plt.figure(figsize=(15, 6))
plt.bar(categories, counts, color="goldenrod")
plt.title('How many in each category')
plt.xlabel('Categories')
plt.ylabel('Count')

# 読みやすくするためにX軸ラベルを回転させます
plt.xticks(rotation=30, ha='right')

# 明確にするために、各バーの上に値ラベルを追加します
for i, v in enumerate(counts):
    plt.text(i, v, f"{v:,}", ha='center', va='bottom')

# チャートを表示します
plt.show()

In [ ]:
# キーが丸い価格であり、価値がその価格のアイテムのリストである辞書を作成します
# これは、サンプリングの価格でアイテムをグループ化するために行われます
slots = defaultdict(list)
for item in items:
    slots[round(item.price)].append(item)

In [ ]:
# 価格のより均一な分布と「自動車」カテゴリへのバイアスを削減したキュレートサンプルデータセットを作成する
# 価格のアイテム> = $ 240は完全に含まれています
# 価格<$ 240の場合、アイテムの数が<= 1200の場合、すべてが含まれています
# 1200を超えるアイテムの数がある場合、1200アイテムの加重ランダムサンプルが採取されます、
# 非自動型アイテムにより高い重量を与える

# 再現性のためにランダムシードを設定します
np.random.seed(42)
random.seed(42)
sample = []
for i in range(1, 1000):
    slot = slots[i]
    if i>=240:
        sample.extend(slot)
    elif len(slot) <= 1200:
        sample.extend(slot)
    else:
        # 重みを割り当てます：「自動車」の場合は1、他のカテゴリには5
        weights = np.array([1 if item.category=='Automotive' else 5 for item in slot])
        # 重みを正規化します
        weights = weights / np.sum(weights)
        # 重みに基づいて1200インデックスをランダムに選択します
        selected_indices = np.random.choice(len(slot), size=1200, replace=False, p=weights)
        # 選択したインデックスに対応するアイテムを選択します
        selected = [slot[i] for i in selected_indices]
        sample.extend(selected)

# キュレーションされたサンプルにアイテムの総数を印刷する
print(f"There are {len(sample):,} items in the sample")

In [ ]:
# キュレーションされたサンプルから価格を抽出します
prices = [float(item.price) for item in sample]
# サンプルデータセットの価格のヒストグラムを作成して表示する
# これにより、サンプリングプロセスが価格分布に及ぼす影響を視覚化するのに役立ちます
plt.figure(figsize=(15, 10))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

In [ ]:
# キュレーションされたサンプルの各カテゴリの発生をカウントします
category_counts = Counter()
for item in sample:
    category_counts[item.category]+=1

# カテゴリとプロットのカウントを抽出します
categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

# サンプル内のカテゴリごとのアイテムのカウントを示すバーチャートを作成して表示します
# これは、カテゴリの分布に対する加重サンプリングの効果を視覚化するのに役立ちます
plt.figure(figsize=(15, 6))
plt.bar(categories, counts, color="lightgreen")

# チャートをカスタマイズします
plt.title('How many in each category')
plt.xlabel('Categories')
plt.ylabel('Count')

# 読みやすくするためにX軸ラベルを回転させます
plt.xticks(rotation=30, ha='right')

# 明確にするために、各バーの上に値ラベルを追加します
for i, v in enumerate(counts):
    plt.text(i, v, f"{v:,}", ha='center', va='bottom')

# チャートを表示します
plt.show()

In [ ]:
# サンプル内のカテゴリ間のアイテムの分布の割合を示すパイチャートを作成して表示します
plt.figure(figsize=(12, 10))
plt.pie(counts, labels=categories, autopct='%1.0f%%', startangle=90)

# 中央に円を追加してドーナツチャートを作成します（オプション）
centre_circle = plt.Circle((0,0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title('Categories')

# 平等なアスペクト比は、パイが円として描かれることを保証します
plt.axis('equal')

plt.show()

In [ ]:
# マークダウンセルは、データセットのキュレーションが完了しており、最終チェックの準備ができていることを示しています
# データセットキュレーション！

# 優れたデータセットを作成しました。

# いくつかの最終チェックをしましょう

In [ ]:
# キュレーションされたサンプルからプロンプトの長さ（文字カウント）と価格を抽出する
sizes = [len(item.prompt) for item in sample]
prices = [item.price for item in sample]

# プロンプトのサイズと価格の関係を視覚化するための散布図を作成して表示します
# これは、2つの間の単純な相関を確認するのに役立ちます
plt.figure(figsize=(15, 8))
plt.scatter(sizes, prices, s=0.2, color="red")

# ラベルとタイトルを追加します
plt.xlabel('Size')
plt.ylabel('Price')
plt.title('Is there a simple correlation?')

# プロットを表示します
plt.show()

In [ ]:
# ヘルパー関数を定義して、アイテムに関する情報を報告する
# アイテムのプロンプト、最後の10トークンID、およびデコードされた最後の10トークンを印刷します
def report(item):
    prompt = item.prompt
    tokens = Item.tokenizer.encode(item.prompt)
    print(prompt)
    print(tokens[-10:])
    print(Item.tokenizer.batch_decode(tokens[-10:]))

In [ ]:
# レポート関数を使用して、サンプル内の特定のアイテムに関する情報を表示します
# これにより、データとトークナー剤の動作を検査するのに役立ちます
report(sample[398000])

## 観察

Llamaトークナイザーの興味深い点は、GPT-4oで見たように、1〜999のすべての数値が1トークンにマッピングされることです。同じことは、すべての数字をトークンにマッピングするQWEN2、GEMMA、およびPHI3にも当てはまりません。これは、私たちのプロジェクトにとって少し便利であることが判明しますが、重要な要件ではありません。

＃ ついに

データをトレーニング、テスト、検証データセットに分割する時が来ました。

テスト目的でデータの5％〜10％を使用することが典型的ですが、実際にはこの時点で必要なものよりもはるかに多くあります。トレーニングには400,000ポイントを獲得し、テスト用に2,000ポイントを予約しますが、すべてを使用することはありません。


In [ ]:
# サンプルをシャッフルして分割する前に、再現性のためにランダムシードを設定します
random.seed(42)
# キュレーションされたサンプルデータセットをシャッフルします
random.shuffle(sample)
# シャッフルサンプルをトレーニング（400,000アイテム）とテスト（2,000アイテム）セットに分割します
train = sample[:400_000]
test = sample[400_000:402_000]
# トレーニングとテストセットのサイズを印刷します
print(f"Divided into a training set of {len(train):,} items and test set of {len(test):,} items")

In [ ]:
# テストセットの最初の250アイテムから価格を抽出します
prices = [float(item.price) for item in test[:250]]
# 最初の250のテスト項目の価格のヒストグラムを作成して表示する
# これにより、テストデータのごく一部の価格分布を簡単に確認できます。
plt.figure(figsize=(15, 6))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

In [ ]:
# トレーニングセットからプロンプトを抽出します
train_prompts = [item.prompt for item in train]
# トレーニングセットから価格を抽出します
train_prices = [item.price for item in train]
# テストセットからテストプロンプト（test_promptメソッドを使用）を抽出する
test_prompts = [item.test_prompt() for item in test]
# テストセットから価格を抽出します
test_prices = [item.price for item in test]

In [ ]:
# トレーニングおよびテストデータから抱き合うフェイスデータセットオブジェクトを作成する
train_dataset = Dataset.from_dict({"text": train_prompts, "price": train_prices})
test_dataset = Dataset.from_dict({"text": test_prompts, "price": test_prices})
# トレーニングおよびテストデータセットを含むデータセットディクションを作成します
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
# 作成されたDataSetDictを抱きしめるフェイスハブにプッシュします
# 「アーロン」を抱きしめる顔のユーザー名に「アーロン」を置き換えます
# データセットには「Your-Username/Pricer-Data」という名前が付けられ、プライベートになります
# hf_user = "aaron-official"＃uncommentとhfユーザー名に置き換えます
# dataset_name = f "{hf_user}/pricer-data"＃uncomment
# dataset.push_to_hub（dataset_name、private = true）＃ハブにプッシュするuncomment

In [ ]:
# ピクルス（シリアル化）トレーニングおよびテストデータセットをファイルとして保存します
# これにより、将来のセッションで処理されたデータをすばやく読み込むことができます
with open('train.pkl', 'wb') as file:
    pickle.dump(train, file)

with open('test.pkl', 'wb') as file:
    pickle.dump(test, file)

In [ ]:
# Googleドライブをマウントして、ドライブ内のファイルにアクセスします
from google.colab import drive
drive.mount('/content/drive')

Googleドライブがマウントされたら、ドライブのフォルダーにファイルをコピーできます。 「私のドライブ/your_folder_name」を、ファイルを保存するフォルダーへのパスに置き換えます。

In [ ]:
# ファイル操作のためにShotilモジュールをインポートします
import shutil

# Googleドライブの宛先パスと、ピクルストレーニングデータのソースパスを定義します
# Googleドライブの目的のフォルダーパスに「My Drive/Your_folder_name」を交換してください
destination_path = '/content/drive/My Drive/train.pkl'
source_path = '/content/train.pkl'

# Colab環境からGoogleドライブに漬けられたトレーニングデータファイルをコピーする
shutil.copyfile(source_path, destination_path)

# 確認メッセージを印刷します
print(f"Copied {source_path} to {destination_path}")

In [ ]:
# ファイル操作のためにShotilモジュールをインポートします
import shutil

# Googleドライブの宛先パスとピクルステストデータのソースパスを定義する
# Googleドライブの目的のフォルダーパスに「My Drive/Your_folder_name」を交換してください
destination_path = '/content/drive/My Drive/test.pkl'
source_path = '/content/test.pkl'

# Colab環境からGoogleドライブに漬物のテストデータファイルをコピーする
shutil.copyfile(source_path, destination_path)

# 確認メッセージを印刷します
print(f"Copied {source_path} to {destination_path}")